In [1]:
import kaggle_environments
from kaggle_environments import get_episode_replay, list_episodes, list_episodes_for_team, list_episodes_for_submission
import json
from kaggle_environments import *
from kaggle_environments.envs.halite.helpers import *
from trainer.policy import *
from trainer.agent import *
from trainer.data_parser import *
import tensorboard
import collections
import time
from trainer.utils import *

data_parser = DataParser()

In [2]:
replay_path = "/data/train/"
meta_path = "/data/meta/"
batch_size = 4

files = os.listdir(replay_path)
agent_ids = {}
with open(meta_path+'agent_ids_5234835.txt') as readfile:
    agent_ids = json.load(readfile)

# trajectory = data_parser.get_trajectory_frames(replay_path+files[2], agent_ids[files[2].split('_')[1][:-4]], one_hot=True)
start = time.perf_counter()
batch = []
fail_count = 0
for i in range(4):
  batch.append(data_parser.get_trajectory_frames(replay_path+files[i], agent_ids[files[i].split('_')[1][:-4]], one_hot=True))

batch = stack_namedtuple(batch, batch_size=4)
end = time.perf_counter()

print(end - start)

FileNotFoundError: [Errno 2] No such file or directory: '/../data/train/convexOptimization_1874236.txt'

In [3]:
print(files[1])
print(files[1].split('_')[1][:-4])

convexOptimization_1834612.txt
1834612


In [30]:
print(f'Map: {trajectory.map.shape}')
print(f'My ships: {trajectory.my_ships.shape}')
print(f'My shipyards: {trajectory.my_shipyards.shape}')
print(f'Close opps: {trajectory.close_opps_obs.shape}')
print(f'Far opp: {trajectory.far_opp_obs.shape}')
print(f'Z: {trajectory.z.shape}')
print(f'Shipyards moves: {trajectory.shipyard_moves.shape}')

Map: (399, 32, 32, 3)
My ships: (399, 32, 32, 3)
My shipyards: (399, 32, 32, 3)
Close opps: (399, 4, 32, 32, 3)
Far opp: (399, 2, 32, 32, 3)
Z: (399, 100, 4)
Shipyards moves: (399, 21, 21, 2)


In [5]:
from policy import *

In [6]:
encoder = Encoder()
latent_game_state, ship_embed, shipyard_embed = encoder(trajectory)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended

In [7]:
encoder.trainable_variables[4]

<tf.Variable 'encoder/spatial_encoder/conv_block/conv2d_1/kernel:0' shape=(3, 3, 4, 4) dtype=float32, numpy=
array([[[[ 0.17906576, -0.2677251 ,  0.05507907,  0.18970343],
         [-0.12011635,  0.02383435, -0.08535795,  0.20401576],
         [ 0.23679042, -0.2219578 ,  0.11787629, -0.14041713],
         [-0.08875814, -0.140919  , -0.10277617, -0.25337046]],

        [[ 0.07666707, -0.00780702, -0.24039617, -0.19186267],
         [-0.19769502, -0.00642267, -0.0904637 ,  0.12281114],
         [ 0.02575195, -0.07967654,  0.14989936, -0.27911678],
         [ 0.08213395, -0.14990294,  0.00868523,  0.01102144]],

        [[-0.01312965, -0.10211572,  0.18276334,  0.14971882],
         [-0.11556602,  0.26035196, -0.0972892 , -0.13853833],
         [-0.02335063,  0.0352008 , -0.21982256,  0.22031361],
         [ 0.0227696 ,  0.25513363,  0.21645832, -0.03852102]]],


       [[[ 0.17870677, -0.09260301, -0.18516031,  0.20399037],
         [-0.2307281 ,  0.2081691 ,  0.17509604, -0.27244985],
 

In [8]:
print(f'Latent: {latent_game_state.shape}')
print(f'Ship embed: {ship_embed[0].shape}')
print(f'Shipyard embed: {ship_embed[2].shape}')

Latent: (399, 128)
Ship embed: (399, 28, 28, 4)
Shipyard embed: (399, 20, 20, 16)


In [9]:
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spatial_encoder (SpatialEnco multiple                  4852      
_________________________________________________________________
spatial_encoder_1 (SpatialEn multiple                  4852      
_________________________________________________________________
spatial_encoder_2 (SpatialEn multiple                  4852      
_________________________________________________________________
spatial_encoder_3 (SpatialEn multiple                  4852      
_________________________________________________________________
spatial_encoder_4 (SpatialEn multiple                  4852      
_________________________________________________________________
scalar_encoder (ScalarEncode multiple                  1400      
_________________________________________________________________
dense (Dense)                multiple                  7449

In [10]:
core = Core(128, return_state=False)
latent_game_state = tf.expand_dims(latent_game_state, 0)
core_output = core(latent_game_state)

In [11]:
print(core_output.shape)

(1, 399, 128)


In [12]:
upsample = Upsample()

In [13]:
# upsample the core out to be later fed to the Conv2dTranspose
shape = core_output.shape[0:2] + [1,1] + core_output.shape[-1]
core_output = tf.reshape(core_output, shape=shape)
core_output = tf.squeeze(core_output, axis=0)
core_output = upsample(core_output)
core_output.shape

TensorShape([399, 16, 16, 32])

In [14]:
ship_decoder = Decoder()
ship_decode = ship_decoder(core_output, ship_embed)
ship_decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
att_up_conv_block (AttUpConv multiple                  20453     
_________________________________________________________________
att_up_conv_block_1 (AttUpCo multiple                  12341     
_________________________________________________________________
att_up_conv_block_2 (AttUpCo multiple                  3117      
Total params: 35,911
Trainable params: 35,793
Non-trainable params: 118
_________________________________________________________________


In [15]:
shipyard_decoder = Decoder()
shipyard_decode = shipyard_decoder(core_output, shipyard_embed)
shipyard_decoder.summary()

Model: "decoder_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
att_up_conv_block_3 (AttUpCo multiple                  20453     
_________________________________________________________________
att_up_conv_block_4 (AttUpCo multiple                  12341     
_________________________________________________________________
att_up_conv_block_5 (AttUpCo multiple                  3117      
Total params: 35,911
Trainable params: 35,793
Non-trainable params: 118
_________________________________________________________________


In [16]:
head = Head()

In [17]:
agent_out = head(ship_decode, shipyard_decode)

In [18]:
print(agent_out[0].shape)
print(agent_out[1].shape)

(399, 21, 21, 6)
(399, 21, 21, 2)


In [19]:
from utils import *

In [20]:
x = tf.random.uniform(shape=(32,22,15))
x = make_time_major(x)
x.shape

TensorShape([22, 32, 15])

In [21]:
x = merge_leading_dim(x, 2)
x.shape

TensorShape([704, 15])

In [22]:
x = split_leading_dim(x, 22,32)
x.shape

TensorShape([22, 32, 15])

In [23]:
x = make_batch_major(x)
x.shape

TensorShape([32, 22, 15])

In [24]:
from agent import Agent

In [25]:
agent = Agent()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended

In [26]:
trajectories = tf.nest.map_structure(lambda t: tf.expand_dims(t, 0), trajectory)
trajectories = make_time_major(trajectories)
trajectories[0].shape

TensorShape([399, 1, 32, 32, 3])

In [27]:
agent_out = agent(trajectories, True)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Batch size: 1
Step size: 399


In [28]:
agent.summary()

Model: "agent"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_1 (Encoder)          multiple                  7475388   
_________________________________________________________________
core_1 (Core)                multiple                  131584    
_________________________________________________________________
upsample_1 (Upsample)        multiple                  164320    
_________________________________________________________________
decoder_2 (Decoder)          multiple                  35911     
_________________________________________________________________
decoder_3 (Decoder)          multiple                  35911     
_________________________________________________________________
head_1 (Head)                multiple                  520       
Total params: 7,843,634
Trainable params: 7,842,598
Non-trainable params: 1,036
_______________________________________________

In [29]:
trajectories.ship_moves.shape

TensorShape([399, 1, 21, 21, 6])